In [2]:
import xarray as xr
import xvec
import matplotlib.pyplot as plt
import geopandas as gpd
from dask.diagnostics import ProgressBar

In [8]:
ds = xr.open_dataset(r"C:\Users\coach\myfiles\postdoc\Invasives\AVIRIS-NG_BIOSCAPE_V02_L3_32_17_RFL.nc", engine="rasterio", chunks="auto")
ds

<xarray.Dataset> Size: 7GB
Dimensions:              (wavelength: 425, x: 2000, y: 2000, band: 1)
Coordinates:
  * wavelength           (wavelength) float64 3kB 377.2 382.2 ... 2.501e+03
  * x                    (x) float64 16kB 8.5e+05 8.5e+05 ... 8.6e+05 8.6e+05
  * y                    (y) float64 16kB 8.7e+05 8.7e+05 ... 8.6e+05 8.6e+05
    transverse_mercator  int64 8B ...
  * band                 (band) int64 8B 1
Data variables: (12/13)
    reflectance          (wavelength, y, x) float32 7GB dask.array<chunksize=(8, 2000, 2000), meta=np.ndarray>
    cosine_i             (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    utc_time             (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    earth_sun_distance   (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    scene_id             (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    path_length          (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    ...                   ...
    to_sensor_zenith     (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    to_sun_azimuth       (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    to_sun_zenith        (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    solar_phase          (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    slope                (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    aspect               (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
Attributes: (12/22)
    Conventions:                       CF-1.6
    creator_name:                      Jet Propulsion Laboratory/California I...
    creator_url:                       aviris.jpl.nasa.gov
    date_created:                      2025-04-23T20:50:10Z
    grid_cell:                         3217
    identifier_product_doi_authority:  https://doi.org
    ...                                ...
    sensor:                            Airborne Visible / Infrared Imaging Sp...
    software_build_version:            2
    summary:                           Mosaic of AVIRIS-NG L2A Reflectance da...
    time_coverage_end:                 2024-11-26T23:59:59Z
    time_coverage_start:               2023-10-22T00:00:00Z
    title:                             AVIRIS-NG L3 Mosaiced Surface Reflecta...

In [9]:
# Load invasive species point data
gdf = gpd.read_file(r'C:\Users\coach\myfiles\postdoc\Invasives\code\hypinvalimap\data\aliens_sep2018.shp')
gdf.head()

,fid,class,group,layer,path,geometry
0,1.0,0,2,point,/Users/glennmoncrieff/Documents/qgis/point.gpk...,POINT (19.16467 -33.99555)
1,2.0,0,2,point,/Users/glennmoncrieff/Documents/qgis/point.gpk...,POINT (19.14254 -33.99885)
2,3.0,0,2,point,/Users/glennmoncrieff/Documents/qgis/point.gpk...,POINT (19.15589 -33.98977)
3,4.0,0,2,point,/Users/glennmoncrieff/Documents/qgis/point.gpk...,POINT (19.13976 -33.99951)
4,5.0,0,2,point,/Users/glennmoncrieff/Documents/qgis/point.gpk...,POINT (19.15018 -33.99258)


In [35]:
def extract_points(ds, points):
    """
    Extracts data values at specified points from a locally stored dataset.

    Parameters:
    - ds: str, path to the dataset (e.g., netCDF or GeoTIFF).
    - points: GeoDataFrame, point locations to extract data.

    Returns:
    - DataFrame containing extracted data values and point indices.
    """
    # Extract data at points
    extracted = ds.xvec.extract_points(
        points['geometry'], 
        x_coords="x", 
        y_coords="y", 
        index=True
    )
    
    return extracted

df = extract_points(ds, gdf)
df

<xarray.Dataset> Size: 3MB
Dimensions:              (wavelength: 425, geometry: 1688, band: 1, index: 1688)
Coordinates:
  * wavelength           (wavelength) float64 3kB 377.2 382.2 ... 2.501e+03
    transverse_mercator  int64 8B ...
  * band                 (band) int64 8B 1
  * geometry             (geometry) object 14kB POINT (19.164674860894667 -33...
    index                (geometry) int64 14kB 0 1 2 3 4 ... 1684 1685 1686 1687
Data variables: (12/15)
    reflectance          (wavelength, geometry) float32 3MB dask.array<chunksize=(8, 1688), meta=np.ndarray>
    cosine_i             (band, geometry) float32 7kB dask.array<chunksize=(1, 1688), meta=np.ndarray>
    utc_time             (band, geometry) float32 7kB dask.array<chunksize=(1, 1688), meta=np.ndarray>
    earth_sun_distance   (band, geometry) float32 7kB dask.array<chunksize=(1, 1688), meta=np.ndarray>
    scene_id             (band, geometry) float32 7kB dask.array<chunksize=(1, 1688), meta=np.ndarray>
    path_length          (band, geometry) float32 7kB dask.array<chunksize=(1, 1688), meta=np.ndarray>
    ...                   ...
    to_sun_zenith        (band, geometry) float32 7kB dask.array<chunksize=(1, 1688), meta=np.ndarray>
    solar_phase          (band, geometry) float32 7kB dask.array<chunksize=(1, 1688), meta=np.ndarray>
    slope                (band, geometry) float32 7kB dask.array<chunksize=(1, 1688), meta=np.ndarray>
    aspect               (band, geometry) float32 7kB dask.array<chunksize=(1, 1688), meta=np.ndarray>
    class                (index) int64 14kB 0 0 0 0 0 0 0 ... 10 10 10 10 10 10
    group                (index) int64 14kB 2 2 2 2 2 2 2 2 ... 2 2 2 2 2 1 0 2
Indexes:
    geometry  GeometryIndex (crs=EPSG:4326)
Attributes: (12/22)
    Conventions:                       CF-1.6
    creator_name:                      Jet Propulsion Laboratory/California I...
    creator_url:                       aviris.jpl.nasa.gov
    date_created:                      2025-04-23T20:50:10Z
    grid_cell:                         3217
    identifier_product_doi_authority:  https://doi.org
    ...                                ...
    sensor:                            Airborne Visible / Infrared Imaging Sp...
    software_build_version:            2
    summary:                           Mosaic of AVIRIS-NG L2A Reflectance da...
    time_coverage_end:                 2024-11-26T23:59:59Z
    time_coverage_start:               2023-10-22T00:00:00Z
    title:                             AVIRIS-NG L3 Mosaiced Surface Reflecta...

In [36]:
# Now that df is extracted, release ds
del ds
import gc
gc.collect()

2890

In [ ]:
df.xvec.to_geodataframe().head()import matplotlib.pyplot as plt
import numpy as np

# Assuming prediction_probabilities is your array of shape (844, 11)
# If it's not already a numpy array, you might need to convert it:
# prediction_probabilities = np.array(prediction_probabilities)

n_samples, n_classes = prediction_probabilities.shape

# Determine grid size for subplots (e.g., 4 rows, 3 columns for 11 classes)
n_cols = 3
n_rows = (n_classes + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 3))
axes = axes.flatten() # Flatten the 2D array of axes for easy iteration

for i in range(n_classes):
    ax = axes[i]
    ax.hist(prediction_probabilities[:, i], bins=20, edgecolor='black')
    ax.set_title(f'Class {i+1}')
    ax.set_xlabel('Probability')
    ax.set_ylabel('Frequency')
    ax.set_xlim(0, 1) # Probabilities are between 0 and 1

# Hide any unused subplots
for j in range(n_classes, len(axes)):
    fig.delaxes(axes[j])

fig.suptitle('Distribution of Class Probabilities', y=1.02, fontsize=16) # Add a main title
plt.tight_layout() # Adjust layout to prevent overlapping titles/labels
plt.show()
import matplotlib.pyplot as plt
import numpy as np

# Assuming prediction_probabilities is your array of shape (844, 11)
# If it's not already a numpy array, you might need to convert it:
# prediction_probabilities = np.array(prediction_probabilities)

n_samples, n_classes = prediction_probabilities.shape

# Determine grid size for subplots (e.g., 4 rows, 3 columns for 11 classes)
n_cols = 3
n_rows = (n_classes + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 3))
axes = axes.flatten() # Flatten the 2D array of axes for easy iteration

for i in range(n_classes):
    ax = axes[i]
    ax.hist(prediction_probabilities[:, i], bins=20, edgecolor='black')
    ax.set_title(f'Class {i+1}')
    ax.set_xlabel('Probability')
    ax.set_ylabel('Frequency')
    ax.set_xlim(0, 1) # Probabilities are between 0 and 1

# Hide any unused subplots
for j in range(n_classes, len(axes)):
    fig.delaxes(axes[j])

fig.suptitle('Distribution of Class Probabilities', y=1.02, fontsize=16) # Add a main title
plt.tight_layout() # Adjust layout to prevent overlapping titles/labels
plt.show()


MemoryError: Unable to allocate 4.51 GiB for an array with shape (425, 1688, 1, 1688) and data type float32

In [11]:
# Add class and group columns to spectral signatures
class_xr = gdf[['class','group']].to_xarray()
ds = ds.merge(class_xr.astype(int),join='left')
ds

<xarray.Dataset> Size: 7GB
Dimensions:              (wavelength: 425, x: 2000, y: 2000, band: 1,
                          index: 1688)
Coordinates:
  * wavelength           (wavelength) float64 3kB 377.2 382.2 ... 2.501e+03
  * x                    (x) float64 16kB 8.5e+05 8.5e+05 ... 8.6e+05 8.6e+05
  * y                    (y) float64 16kB 8.7e+05 8.7e+05 ... 8.6e+05 8.6e+05
    transverse_mercator  int64 8B ...
  * band                 (band) int64 8B 1
  * index                (index) int64 14kB 0 1 2 3 4 ... 1684 1685 1686 1687
Data variables: (12/15)
    reflectance          (wavelength, y, x) float32 7GB dask.array<chunksize=(8, 2000, 2000), meta=np.ndarray>
    cosine_i             (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    utc_time             (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    earth_sun_distance   (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    scene_id             (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    path_length          (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    ...                   ...
    to_sun_zenith        (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    solar_phase          (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    slope                (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    aspect               (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    class                (index) int64 14kB 0 0 0 0 0 0 0 ... 10 10 10 10 10 10
    group                (index) int64 14kB 2 2 2 2 2 2 2 2 ... 2 2 2 2 2 1 0 2
Attributes: (12/22)
    Conventions:                       CF-1.6
    creator_name:                      Jet Propulsion Laboratory/California I...
    creator_url:                       aviris.jpl.nasa.gov
    date_created:                      2025-04-23T20:50:10Z
    grid_cell:                         3217
    identifier_product_doi_authority:  https://doi.org
    ...                                ...
    sensor:                            Airborne Visible / Infrared Imaging Sp...
    software_build_version:            2
    summary:                           Mosaic of AVIRIS-NG L2A Reflectance da...
    time_coverage_end:                 2024-11-26T23:59:59Z
    time_coverage_start:               2023-10-22T00:00:00Z
    title:                             AVIRIS-NG L3 Mosaiced Surface Reflecta...

In [26]:
ds.reflectance.data.shape

(425, 2000, 2000)

In [12]:
with ProgressBar():
    dsp = ds.persist()

[########################################] | 100% Completed | 338.30 s


In [23]:
dsp.isnull().sum(dim='wavelength').compute()  # Check for NaN values in the dataset

<xarray.Dataset> Size: 80MB
Dimensions:              (y: 2000, x: 2000, band: 1, index: 1688)
Coordinates:
  * x                    (x) float64 16kB 8.5e+05 8.5e+05 ... 8.6e+05 8.6e+05
  * y                    (y) float64 16kB 8.7e+05 8.7e+05 ... 8.6e+05 8.6e+05
    transverse_mercator  int64 8B 0
  * band                 (band) int64 8B 1
  * index                (index) int64 14kB 0 1 2 3 4 ... 1684 1685 1686 1687
Data variables: (12/15)
    reflectance          (y, x) int64 32MB 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0
    cosine_i             (band, y, x) bool 4MB False False False ... False False
    utc_time             (band, y, x) bool 4MB False False False ... False False
    earth_sun_distance   (band, y, x) bool 4MB False False False ... False False
    scene_id             (band, y, x) bool 4MB False False False ... False False
    path_length          (band, y, x) bool 4MB False False False ... False False
    ...                   ...
    to_sun_zenith        (band, y, x) bool 4MB False False False ... False False
    solar_phase          (band, y, x) bool 4MB False False False ... False False
    slope                (band, y, x) bool 4MB False False False ... False False
    aspect               (band, y, x) bool 4MB False False False ... False False
    class                (index) bool 2kB False False False ... False False
    group                (index) bool 2kB False False False ... False False

In [21]:
dsp.dropna(dim='index', subset=['reflectance'])

<xarray.Dataset> Size: 7GB
Dimensions:              (wavelength: 425, x: 2000, y: 2000, band: 1,
                          index: 1688)
Coordinates:
  * wavelength           (wavelength) float64 3kB 377.2 382.2 ... 2.501e+03
  * x                    (x) float64 16kB 8.5e+05 8.5e+05 ... 8.6e+05 8.6e+05
  * y                    (y) float64 16kB 8.7e+05 8.7e+05 ... 8.6e+05 8.6e+05
    transverse_mercator  int64 8B ...
  * band                 (band) int64 8B 1
  * index                (index) int64 14kB 0 1 2 3 4 ... 1684 1685 1686 1687
Data variables: (12/15)
    reflectance          (wavelength, y, x) float32 7GB dask.array<chunksize=(8, 2000, 2000), meta=np.ndarray>
    cosine_i             (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    utc_time             (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    earth_sun_distance   (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    scene_id             (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    path_length          (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    ...                   ...
    to_sun_zenith        (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    solar_phase          (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    slope                (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    aspect               (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    class                (index) int64 14kB 0 0 0 0 0 0 0 ... 10 10 10 10 10 10
    group                (index) int64 14kB 2 2 2 2 2 2 2 2 ... 2 2 2 2 2 1 0 2
Attributes: (12/22)
    Conventions:                       CF-1.6
    creator_name:                      Jet Propulsion Laboratory/California I...
    creator_url:                       aviris.jpl.nasa.gov
    date_created:                      2025-04-23T20:50:10Z
    grid_cell:                         3217
    identifier_product_doi_authority:  https://doi.org
    ...                                ...
    sensor:                            Airborne Visible / Infrared Imaging Sp...
    software_build_version:            2
    summary:                           Mosaic of AVIRIS-NG L2A Reflectance da...
    time_coverage_end:                 2024-11-26T23:59:59Z
    time_coverage_start:               2023-10-22T00:00:00Z
    title:                             AVIRIS-NG L3 Mosaiced Surface Reflecta...

In [17]:
dsp.xvec.to_geodataframe()

MemoryError: Unable to allocate 10.4 TiB for an array with shape (425, 2000, 2000, 1, 1688) and data type float32